# File to generate Nadir OSSE tracks 

In [1]:
import numpy as np 
import xarray as xr
import glob 
import os 
from scipy import interpolate
import random
import datetime

import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy

import gc

from joblib import Parallel
from joblib import delayed as jb_delayed


Choice of the OSSE zone between *crossover_CCS*, *crossover_centerpacific* and *crossover_hawaii*.

In [2]:
### ZONE OF CROSSOVERS ###

#zone = "crossover_CCS"
#zone = "crossover_centerpacific"
#zone = "crossover_hawaii"

### ZONE OF IMAGINARY CROSSOVERS ### 

zone = "hawaii"

Choice of the variables to integrate in the interpolated measurements among : 
- *ssh_cor* : Total DAC corrected SSH, renamed *ssh* in the output dataset 
- *ssh_bm* : Balanced Motion,
- *ssh_it1* : 1st Mode Internal Tide, 
- *ssh_it2* : 2nd Mode Internal Tide, 
- *ssh_it3* : 3d Mode Internal Tide, 
- *ssh_it_tot* : Total Internal Tide,
- *ssh_it1_coherent* : Coherent part of the 1st Mode Internal Tide. 

In [3]:
list_variables = ["ssh_cor","ssh_bm","ssh_it1"]
dic_attrs = dict({"ssh":"Total DAC corrected SSH.",
                  "ssh_bm":"Balanced Motion.",
                  "ssh_it1":"1st Mode Internal Tide.",
                  "ssh_it2":"2nd Mode Internal Tide.",
                  "ssh_it3":"3d Mode Internal Tide.",
                  "ssh_it_tot":"Total Internal Tide.",
                  "ssh_it1_coherent":"Coherent part of the 1st Mode Internal Tide."})

Definition of the coordinates limits : 

In [4]:
if zone == "crossover_CCS":
    lon_min = 225; lon_max = 245;lat_min = 25; lat_max = 45;
elif zone == "crossover_centerpacific":
    lon_min = 200; lon_max = 220;lat_min = 25; lat_max = 45;
elif zone == "crossover_hawaii":
    lon_min = 180; lon_max = 200;lat_min = 25; lat_max = 45;
elif zone == "hawaii":
    lon_min = 185; lon_max = 205;lat_min = 15; lat_max = 35;

## 1. - Functions 

In [5]:
def open_satellite_data(date,str_satellite):

    dir_input = "/bettik/bellemva/satellite_track"

    date_split = date.astype('str').split("-")

    path_obs = os.path.join(dir_input,str_satellite,"2019","dt_global_*"+date_split[1]+date_split[2]+"*.nc")
    ds = xr.open_mfdataset(path_obs,combine='nested',concat_dim="time")

    lon_nad = ds.longitude
    lat_nad = ds.latitude

    mask = (lon_nad>lon_min)*(lon_nad<lon_max)*(lat_nad>lat_min)*(lat_nad<lat_max)

    ds = ds.where(mask,drop=True)

    empty_array = xr.DataArray(np.empty(ds.time.shape), dims=['time'])

    track = xr.Dataset({'point':(('time'),empty_array.data)},
                       coords={'time':('time',ds.time.values),'latitude':('latitude',ds.latitude.values),'longitude':('longitude',ds.longitude.values)})
    
    new_time_array = np.array([np.datetime64(t.astype('str').replace("2019","2012")).astype('datetime64[s]') for t in track.time.values])
    new_time_array_float = np.array([t.astype('float64') for t in new_time_array])
    track = track.assign_coords({"time":new_time_array_float})

    return track, new_time_array


In [6]:
def open_model_data(date,var,omit_next_step=False):

    if "ssh_it" in var : # variable of internal tide ssh, contained inside MITgcm_it directory 
        dir_input_ssh = f"/bettik/bellemva/MITgcm/MITgcm_it/{zone}/"
        if omit_next_step:
            path = os.path.join(dir_input_ssh,"MITgcm_it_"+date.astype('str').replace("-","")+".nc")
        else : 
            path = []
            path.append(os.path.join(dir_input_ssh,"MITgcm_it_"+date.astype('str').replace("-","")+".nc"))
            path.append(os.path.join(dir_input_ssh,"MITgcm_it_"+(date+np.timedelta64(1,"D")).astype('str').replace("-","")+".nc"))
    else : 
        dir_input_ssh = f"/bettik/bellemva/MITgcm/MITgcm_filtered_final"
        if omit_next_step:
            path = os.path.join(dir_input_ssh,"MITgcm_filt_"+date.astype('str').replace("-","")+".nc")
        else :  
            path = []
            path.append(os.path.join(dir_input_ssh,"MITgcm_filt_"+date.astype('str').replace("-","")+".nc"))
            path.append(os.path.join(dir_input_ssh,"MITgcm_filt_"+(date+np.timedelta64(1,"D")).astype('str').replace("-","")+".nc"))
    if omit_next_step:
        ds_ssh = xr.open_dataset(path)
    else :
        ds_ssh = xr.open_mfdataset(path,combine="nested",concat_dim="time")
        ds_ssh = ds_ssh.sel(longitude = slice(lon_min,lon_max),latitude=slice(lat_min,lat_max),drop=True)

    time_array_float = np.array([t.astype("datetime64[s]").astype('float64') for t in ds_ssh.time.values])
    ds_ssh = ds_ssh.assign_coords({"time":time_array_float})

    if omit_next_step:
        finterp = interpolate.RegularGridInterpolator([ds_ssh.time.values[0:24],ds_ssh.latitude.values,ds_ssh.longitude.values],
                                                ds_ssh[var].values[0:24],
                                                bounds_error=False)
    else : 
        finterp = interpolate.RegularGridInterpolator([ds_ssh.time.values[0:25],ds_ssh.latitude.values,ds_ssh.longitude.values],
                                                ds_ssh[var].values[0:25],
                                                bounds_error=False)
    
    return finterp 

In [7]:
def interp_satellite(track,new_time_array,interp,var):
    
    ssh_interp = interp(np.transpose([track.time.values, track.latitude.values,track.longitude.values]))

    #if var is "ssh_cor", renaming it into "ssh" in the netcdf file 
    if var == "ssh_cor":
        var = "ssh"

    ds = xr.Dataset(data_vars={var:(('time'),ssh_interp[np.invert(np.isnan(ssh_interp))]),}, # don't forget to add 'time',
                    coords={'time':('time',new_time_array[np.invert(np.isnan(ssh_interp))]),
                            'latitude':('time',track.latitude.values[np.invert(np.isnan(ssh_interp))]),
                            'longitude':('time',track.longitude.values[np.invert(np.isnan(ssh_interp))])},
                    )
    
    ds[var] = ds[var].assign_attrs({"description":dic_attrs[var],
                           "units":"[m]"})
    
    return ds 


In [8]:
name_folder = dict({"crossover_CCS":"2023b_SSHmapping_HF_Crossover_California","crossover_centerpacific":"2023c_SSHmapping_HF_Crossover_CenterPacific",\
                    "crossover_hawaii":"2023d_SSHmapping_HF_Crossover_Hawaii","hawaii":"2023e_SSHmapping_HF_Hawaii_new"})

def create_dataset (date,omit_next_step=False):

    for sat in ["alg","c2","j3","s3a","s3b"]:
        
        track, new_time_array = open_satellite_data(date,sat)

        list_ds = []

        for var in list_variables : 
            finterp = open_model_data(date,var,omit_next_step)
            list_ds.append(interp_satellite(track,new_time_array,finterp,var))
            del finterp
        
        ds = xr.merge(list_ds)

        try:
            ds.to_netcdf(f"/bettik/bellemva/ocean_data_challenge/{name_folder[zone]}/dc_obs_nadirs/"+sat+"/SSH_NADIR_"+date.astype('str')+".nc",
                        encoding = {"time" : {'units' : "nanoseconds since 2012-01-01 00:00:00.000000000"}})
        except ValueError:
            print("Na data for "+sat+" in that zone for "+date.astype('str'))

        del track, ds, new_time_array 
    
    print(date,"done")

    return None 

## 2. - Track interpolating 

In [9]:
# dates = np.arange(np.datetime64("2012-05-02"),np.datetime64("2012-08-01"))
dates = np.arange(np.datetime64("2012-06-03"),np.datetime64("2012-06-04"))

for d in dates : 
    omit_next_step = False 
    if d == np.datetime64("2012-07-31"):
        omit_next_step = True
    create_dataset(d,omit_next_step)

/home/bellemva/miniconda3/envs/pangeo-forge/lib/python3.10/site-packages/xarray/backends/plugins.py:71: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


2012-06-03 done


In [ ]:
create_dataset(np.datetime64("2012-07-31"),omit_next_step=True)

In [ ]:
test = xr.open_dataset(f"/bettik/bellemva/ocean_data_challenge/{name_folder[zone]}/dc_obs_nadirs/alg/SSH_NADIR_2012-07-14.nc")
test

plt.scatter(test.longitude,test.latitude,c=test.ssh_bm)
#plt.plot(test.ssh_it1,label="IT")
#plt.plot(test.ssh_bm,label="BM")
#plt.plot(test.ssh,label="TOT")
#plt.plot(test.ssh-test.ssh_bm,label="DIFF")

#plt.legend()


In [ ]:
longitude = np.arange(180,245.01,1/48)
latitude = np.arange(10,45.01,1/48)

fig, ax = plt.subplots(figsize=(9,4),dpi=200,subplot_kw={"projection": ccrs.PlateCarree(central_longitude=0.)})

ax.set_extent([-180, -115,10, 45],crs=ccrs.PlateCarree())
map_diff = ax.scatter(ds_alg.longitude.values,
                      ds_alg.latitude.values,
                      c=ds_alg.ssh_cor.values,cmap="Spectral",vmin=-1,vmax=1)

ax.add_feature(cfeature.LAND,color="lightgrey",zorder=1)
ax.add_feature(cfeature.OCEAN,color="black",zorder=0)
ax.coastlines(lw=0.5)
ax.set_aspect("equal")
ax.set_title("Altika OSSE tracks")

gridlines = ax.gridlines(
    alpha=0.3,
    color="white",
    draw_labels=True,
    xlocs=np.arange(-180,-115,10),
    ylocs=np.arange(10,45,5),
    # xformatter=cartopy.mpl.ticker.LongitudeFormatter(zero_direction_label=True,),
)
gridlines.right_labels = False
gridlines.top_labels  = False
gridlines.xlabel_style = {'size': 11}
gridlines.ylabel_style = {'size': 11}
ax.set_aspect("equal")
